# Machine Learning (Final Project)

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [24]:
plt.style.use("seaborn-v0_8-whitegrid")

In [25]:
df = pd.read_csv('data/Malware_subset.csv')

In [26]:
df.isnull().sum()

 Source Port          0
 Destination Port     0
 Protocol             0
 Flow Duration        0
 Total Fwd Packets    0
                     ..
Idle Mean             0
 Idle Std             0
 Idle Max             0
 Idle Min             0
Label                 0
Length: 81, dtype: int64

In [27]:
df.head()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,43399,443,6,36586,1,1,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ADWARE_FEIWO
1,40422,443,6,36984,1,1,0.0,0.0,0.0,0.0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ADWARE_FEIWO
2,56977,80,6,15638963,5,5,405.0,448.0,405.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ADWARE_FEIWO
3,60652,80,6,15660249,5,5,405.0,448.0,405.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ADWARE_FEIWO
4,44512,80,6,14456391,5,3,268.0,519.0,268.0,0.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ADWARE_FEIWO


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834695 entries, 0 to 834694
Data columns (total 81 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Source Port                  834695 non-null  int64  
 1    Destination Port             834695 non-null  int64  
 2    Protocol                     834695 non-null  int64  
 3    Flow Duration                834695 non-null  int64  
 4    Total Fwd Packets            834695 non-null  int64  
 5    Total Backward Packets       834695 non-null  int64  
 6   Total Length of Fwd Packets   834695 non-null  float64
 7    Total Length of Bwd Packets  834695 non-null  float64
 8    Fwd Packet Length Max        834695 non-null  float64
 9    Fwd Packet Length Min        834695 non-null  float64
 10   Fwd Packet Length Mean       834695 non-null  float64
 11   Fwd Packet Length Std        834695 non-null  float64
 12  Bwd Packet Length Max         834695 non-nul

In [29]:
# Label Encoder
encoder = LabelEncoder()
encoded = encoder.fit_transform(df['Label'])
df['Label'] = encoder.fit_transform(df['Label'])

#Remove duplicated data
df.duplicated().sum()
df = df.drop_duplicates()


# andle missing data
df.isna().sum()

 Source Port          0
 Destination Port     0
 Protocol             0
 Flow Duration        0
 Total Fwd Packets    0
                     ..
Idle Mean             0
 Idle Std             0
 Idle Max             0
 Idle Min             0
Label                 0
Length: 81, dtype: int64

In [30]:
df.groupby('Label')['Label'].count()

Label
0      39679
1      43374
2      56630
3      93767
4      38769
5      32545
6      31031
7      13028
8      39267
9      36030
10    410543
Name: Label, dtype: int64

In [31]:
np.isinf(df).sum().sum()

np.int64(0)

In [32]:
# selected_features = [' Bwd Header Length', ' Fwd Header Length.1', ' Subflow Bwd Packets', ' min_seg_size_forward', 
                    # ' Total Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Std', ' Bwd Packet Length Std',
                    # 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Std', 'Fwd Packets/s', ' Min Packet Length']

# X = df[selected_features]
X = df.drop('Label', axis=1)
y = df["Label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# X_train  #imbalanced 
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

In [34]:
y_train.value_counts()

Label
2     328582
10    328582
9     328582
0     328582
5     328582
3     328582
1     328582
4     328582
7     328582
8     328582
6     328582
Name: count, dtype: int64

In [35]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

print("Selected features:", selected_features)
print("Predictions:", y_pred)

MemoryError: could not allocate 184549376 bytes

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

# model = KNeighborsClassifier(n_neighbors=10)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)


In [ ]:
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred)
print("F1-Score:", f1)
